## Apply all imports here

In [1]:
from os import listdir, getcwd
import os
from os.path import isfile, join
import subprocess
import sys
import csv
import pandas as pd



## Read all csv files 

In [ ]:
mypath = join(getcwd(), 'eval_results/soot_res')
# print(mypath)

csv_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
# print(csv_files)

completepaths = [os.path.join(mypath, f) for f in csv_files]
# print(completepaths)

# desired_col = ["jar","solver","thread","totalRuntime","cgConstructionTime","prop","method","mem","CallGraphAlgo","callGraphEdges","callGraphReachableNodes","initialStmtCount","stmtCountAfterApplyingBI","BodyTransformers","BodyTransformersMetrics"]
desired_col = ["jar","solver","thread","totalRuntime","cgConstructionTime","prop","method","mem","CallGraphAlgo","callGraphEdges","callGraphReachableNodes","initialStmtCount","stmtCountAfterApplyingBI","BodyTransformers","BodyTransformersMetrics"]

for completefilepath in completepaths:
    # print(completefilepath)
    csv_cols = next(csv.reader(open(completefilepath))) # read only the csv columns names

    cols = [col for col in desired_col if col in csv_cols]
    df = pd.read_csv(completefilepath, sep=";", header=0)
    print(df.head())
    print("-----------")

In [ ]:
df.iloc[0]

## Convert csv with , files

In [ ]:
import re
import json
import pandas as pd
import csv

mypath = join(getcwd(), 'eval_results/soot_res')
# print(mypath)

csv_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
# print(csv_files)

completepaths = [os.path.join(mypath, f) for f in csv_files]
# print(completepaths)

for completepath in completepaths:
    with open(completepath, 'r', encoding='UTF-8') as file:
        lines = [line.rstrip() for line in file]

# print(lines)


# input_str = """
# jar,solver,thread,totalRuntime,cgConstructionTime,prop,method,mem,CallGraphAlgo,callGraphEdges,callGraphReachableNodes,initialStmtCount,stmtCountAfterApplyingBI,BodyTransformers,BodyTransformersMetrics
# assertj-core-3.24.2,default,1,88422,5368,3974427,50,554,CHA,57910,13599,101912,102790,[JB_LS, JB_TR],{jb.tr=[1470, -366], jb.ls=[903, -50]}
# assertj-core-3.24.2,default,1,62110,6085,3020975,50,594,CHA,57910,13599,101912,102790,[JB_LS, JB_TR, JB_LP],{jb.tr=[1424, 2], jb.lp=[697, -126], jb.ls=[731, -476]}
# """
input_str = '\n'.join(lines)
print(input_str)

new_ouput = ""
new_different_output = ""
for each_row in input_str.split('\n'): 
    if each_row.strip() != "":
        split_row = each_row.split(',')
        new_ouput += ','.join(split_row[:13])
        new_ouput += '\n'
        new_different_output += ','.join(split_row[13:])
        new_different_output += '\n'

with open('simple_output.csv', 'w') as file:
    file.write(new_ouput)

df = pd.read_csv('simple_output.csv')


pattern = r'^(\[.*?\]),(\{.*\})$'

each_index = 0
for each_row in new_different_output.split('\n'):
    if each_row.strip() != "" and not each_row.startswith('BodyTransformers'):
        match = re.match(pattern, each_row)
        if match:
            BodyTransformers_str = match.group(1)
            column_y = BodyTransformers_str.strip('[]').split(',')
            print(each_index)
            df.loc[each_index, 'BodyTransformers'] = json.dumps(column_y)

            column_z_str = match.group(2).replace('=', ':')
            column_z_str = re.sub(r'([\w\.]+):', r'"\1":', column_z_str)
            column_z = json.loads(column_z_str)
            df.loc[each_index, 'BodyTransformersMetrics'] = json.dumps(column_z)
        each_index += 1

print(df.head())